In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
base = pd.read_csv('bases/games.csv')
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
base = base.drop(['Other_Sales', 'Global_Sales', 'Developer'], axis=1)
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,NaN,NaN,NaN,NaN,NaN


In [5]:
base.shape

(16719, 13)

In [8]:
base.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [14]:
base = base.dropna(axis=0)

In [15]:
base.shape

(6825, 13)

In [16]:
base.isnull().sum()

Name               0
Platform           0
Year_of_Release    0
Genre              0
Publisher          0
NA_Sales           0
EU_Sales           0
JP_Sales           0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

In [18]:
base['Name'].value_counts()

Name
Need for Speed: Most Wanted                  8
Madden NFL 07                                8
LEGO Star Wars II: The Original Trilogy      8
The Sims 2                                   7
Terraria                                     7
                                            ..
Castlevania: Portrait of Ruin                1
Suzuki TT Superbikes                         1
Rumble Roses                                 1
Sherlock Holmes: The Mystery of the Mummy    1
STORM: Frontline Nation                      1
Name: count, Length: 4377, dtype: int64

In [19]:
base = base.drop('Name', axis=1)

In [20]:
base.shape

(6825, 12)

In [22]:
base.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating'],
      dtype='object')

In [23]:
x = base.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values

In [24]:
y_na = base.iloc[:, 4].values
y_eu = base.iloc[:, 5].values
y_jp = base.iloc[:, 6].values

In [26]:
base['Platform'].value_counts()

Platform
PS2     1140
X360     858
PS3      769
PC       651
XB       565
Wii      479
DS       464
PSP      390
GC       348
PS4      239
GBA      237
XOne     159
3DS      155
PS       150
PSV      118
WiiU      89
DC        14
Name: count, dtype: int64

In [28]:
one_hot = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 2, 3, 8])], remainder='passthrough')
x = one_hot.fit_transform(x).toarray()

In [29]:
x.shape

(6825, 303)

## **Rede Neural**

In [30]:
entrada = Input(shape=(x.shape[1],))
oculta1 = Dense(units=153, activation='relu')(entrada)
oculta2 = Dense(units=153, activation='relu')(oculta1)
saida1 = Dense(units=1, activation='linear')(oculta2)
saida2 = Dense(units=1, activation='linear')(oculta2)
saida3 = Dense(units=1, activation='linear')(oculta2)

In [31]:
regressor = Model(inputs=entrada, outputs=[saida1, saida2, saida3])

In [32]:
regressor.compile(optimizer='adam', loss='mse')

In [33]:
regressor.fit(x, [y_na, y_eu, y_jp], epochs=500, batch_size=100)

Epoch 1/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - dense_2_loss: 456.0183 - dense_3_loss: 313.7054 - dense_4_loss: 277.3374 - loss: 1047.2043
Epoch 2/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - dense_2_loss: 6.9470 - dense_3_loss: 4.0463 - dense_4_loss: 0.6629 - loss: 11.6581
Epoch 3/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - dense_2_loss: 3.0594 - dense_3_loss: 1.9930 - dense_4_loss: 2.2779 - loss: 7.3319
Epoch 4/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - dense_2_loss: 5.2816 - dense_3_loss: 4.5437 - dense_4_loss: 2.2380 - loss: 12.0683
Epoch 5/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - dense_2_loss: 2.1812 - dense_3_loss: 1.8093 - dense_4_loss: 1.5121 - loss: 5.5033
Epoch 6/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - dense_2_loss: 1.8103 - dense_3_loss: 1.7977 - dense_4_loss: 0.9541 - loss: 4.5646
Epoch 7/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - dense_2_loss: 19.0201 - dense_3_loss: 21.3198 - dense_4_loss: 14.8315 - loss: 55.0450
Epoch 8/500
69/69 ━━━━━━━━━━━━━━━━━━━━

In [34]:
prevs_na, prevs_eu, prevs_jp = regressor.predict(x)

214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [35]:
prevs_na.mean()

0.38680086

In [36]:
y_na.mean()

0.3944835164835165

In [37]:
from sklearn.metrics import mean_absolute_error

In [39]:
mean_absolute_error(y_na, prevs_na)

0.2823693374857361

In [40]:
mean_absolute_error(y_eu, prevs_eu)

0.20415857281658675

In [41]:
mean_absolute_error(y_jp, prevs_jp)

0.08963409154790661

### **Previsão de Global Sales** 

In [122]:
base = pd.read_csv('bases/games.csv')

In [123]:
base = base.drop(['NA_Sales', 'JP_Sales', 'EU_Sales'], axis=1)
base.shape

(16719, 13)

In [99]:
base.isna().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

In [55]:
base['Name'].value_counts()

Name
Need for Speed: Most Wanted                         12
FIFA 14                                              9
Ratatouille                                          9
LEGO Marvel Super Heroes                             9
Madden NFL 07                                        9
                                                    ..
Jewels of the Tropical Lost Island                   1
Sherlock Holmes and the Mystery of Osborne House     1
The King of Fighters '95 (CD)                        1
Megamind: Mega Team Unite                            1
Haitaka no Psychedelica                              1
Name: count, Length: 11562, dtype: int64

In [124]:
base = base.drop('Name', axis=1)

In [62]:
base['Genre'].value_counts()

Genre
Action          3370
Sports          2348
Misc            1750
Role-Playing    1500
Shooter         1323
Adventure       1303
Racing          1249
Platform         888
Simulation       874
Fighting         849
Strategy         683
Puzzle           580
Name: count, dtype: int64

In [125]:
base = base[base['Genre'].notna()]

In [102]:
base['Year_of_Release'].value_counts()

Year_of_Release
2008.0    1427
2009.0    1426
2010.0    1255
2007.0    1197
2011.0    1136
2006.0    1006
2005.0     939
2002.0     829
2003.0     775
2004.0     762
2012.0     653
2015.0     606
2014.0     581
2013.0     544
2016.0     502
2001.0     482
1998.0     379
2000.0     350
1999.0     338
1997.0     289
1996.0     263
1995.0     219
1994.0     121
1993.0      60
1981.0      46
1992.0      43
1991.0      41
1982.0      36
1986.0      21
1989.0      17
1983.0      17
1990.0      16
1987.0      16
1988.0      15
1985.0      14
1984.0      14
1980.0       9
2017.0       3
2020.0       1
Name: count, dtype: int64

In [72]:
base['Year_of_Release'].mean()

2006.4889956225682

In [126]:
base['Year_of_Release'].fillna(base['Year_of_Release'].mean(), inplace=True)

In [75]:
base['Year_of_Release'].isna().sum()

0

In [127]:
base = base.drop('Developer', axis=1)

**Critic_Score** - 8582 <br>
**Critic_Count** - 8582 <br>
**User_Score** - 6704 <br>
**User_Count** - 9129 <br>
**Rating** - 6769 <br>

In [128]:
base['User_Score'] = pd.to_numeric(base['User_Score'], errors='coerce')

In [129]:
base = base.drop('Rating', axis=1)

In [130]:
base['Critic_Score'].fillna(base['Critic_Score'].mean(), inplace=True)
base['Critic_Count'].fillna(base['Critic_Count'].mean(), inplace=True)
base['User_Score'].fillna(base['User_Score'].mean(), inplace=True)
base['User_Count'].fillna(base['User_Count'].mean(), inplace=True)

In [131]:
base['Publisher'].fillna(base['Publisher'].mode()[0], inplace=True)

In [132]:
base.isna().sum()

Platform           0
Year_of_Release    0
Genre              0
Publisher          0
Other_Sales        0
Global_Sales       0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
dtype: int64

In [133]:
base.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'Other_Sales',
       'Global_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count'],
      dtype='object')

In [138]:
x = base.iloc[:, [0, 1, 2, 3, 4, 6, 7, 8, 9]].values
x.shape

(16717, 9)

In [139]:
y = base.iloc[:, 5].values
y.shape

(16717,)

In [146]:
x[0]

array(['Wii', 2006.0, 'Sports', 'Nintendo', 8.45, 76.0, 51.0, 8.0, 322.0],
      dtype=object)

In [152]:
one_hot = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 2, 3])], remainder='passthrough')
x = one_hot.fit_transform(x).toarray()

In [153]:
x.shape

(16717, 630)

In [154]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [155]:
x_train.shape, y_train.shape

((13373, 630), (13373,))

In [156]:
x_test.shape, y_test.shape

((3344, 630), (3344,))

In [157]:
(630 + 1) / 2

315.5

In [ ]:
# entrada = Input(shape=(x.shape[1],))
# oculta1 = Dense(units=153, activation='relu')(entrada)
# oculta2 = Dense(units=153, activation='relu')(oculta1)
# saida1 = Dense(units=1, activation='linear')(oculta2)
# saida2 = Dense(units=1, activation='linear')(oculta2)
# saida3 = Dense(units=1, activation='linear')(oculta2)

In [160]:
import tensorflow.keras as keras

In [166]:
x.shape[1]

630

In [186]:
entrada = Input(shape=(x.shape[1],))
oculta1 = Dense(units=316, activation=Activation(keras.activations.relu))(entrada)
dropout1 = Dropout(rate=0.2)(oculta1)
oculta2 = Dense(units=316, activation=Activation(keras.activations.relu))(dropout1)
dropout2 = Dropout(rate=0.2)(oculta2)
saida = Dense(units=1, activation='linear')(dropout2)

In [187]:
regressor = Model(inputs=entrada, outputs=saida)
regressor.compile(optimizer='adam', loss='mse')
regressor.fit(x_train, y_train, epochs=500, batch_size=100)

Epoch 1/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 2508.3386
Epoch 2/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 158.8310
Epoch 3/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 50.1756
Epoch 4/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 25.0255
Epoch 5/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 22.5123
Epoch 6/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 7.1329
Epoch 7/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5.6090
Epoch 8/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 4.2849
Epoch 9/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 4.4460
Epoch 10/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.6758
Epoch 11/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 3.3141
Epoch 12/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 4.3083
Epoch 13/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 2.8841
Epoch 14/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.4160
Epoch 15/500
134/134 

In [178]:
regressor.evaluate(x_test, y_test)

105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.2772


2.484065055847168

In [188]:
from sklearn.metrics import mean_absolute_error
prevs = regressor.predict(x_test)
mean_absolute_error(prevs, y_test)

105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


0.48741766307556467